In [1]:
import pandas as pd
import pandas_profiling as pp

/var/folders/py/gsn_r2vn4zj7d6j3f2p58mcw0000gn/T/ipykernel_4322/120689590.py:2: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling as pp


In [2]:
df = pd.read_csv("/Users/ugo/Documents/challenge_data_pacific/data_nc/prix-des-produits-alimentaires-en-2023-pacific-dataviz-challenge.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4363989 entries, 0 to 4363988
Data columns (total 30 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   CODE_PRODUIT              int64  
 1   NOM                       object 
 2   SECTEUR_CONSO             object 
 3   SOUS_SECTEUR_CONSO        object 
 4   FAMILLE_PRODUIT           object 
 5   SOUS_FAMILLE              object 
 6   VARIETE                   object 
 7   CATEGORIE                 object 
 8   DEPARTEMENT               object 
 9   MARQUE                    object 
 10  CONDITIONNEMENT           float64
 11  COND_UNITE_LIBELLE_LONG   object 
 12  COND_UNITE_LIBELLE_COURT  object 
 13  GENCOD                    object 
 14  PRIX_RELEVE               float64
 15  DATE_RELEVE               object 
 16  PROMOTION                 int64  
 17  TOP_500                   object 
 18  TYPE_RELEVE               object 
 19  RAISON_SOCIALE            object 
 20  RIDET                   

In [7]:
report = pp.ProfileReport(df[['CODE_PRODUIT',
 'NOM',
 'SECTEUR_CONSO',
 'SOUS_SECTEUR_CONSO',
 'FAMILLE_PRODUIT',
 'SOUS_FAMILLE',
 'VARIETE',
 'CATEGORIE',
 'DEPARTEMENT',
 'MARQUE',
 'CONDITIONNEMENT',
 'COND_UNITE_LIBELLE_LONG',
 'COND_UNITE_LIBELLE_COURT',
 'GENCOD',
 'PRIX_RELEVE',
 'DATE_RELEVE',
 'PROMOTION',
 'TOP_500',
 'TYPE_RELEVE',
 'RAISON_SOCIALE',
 'RIDET',
 'COMMUNE']], minimal=True)
report.to_file("report.html")

/Users/ugo/Documents/pacific-dataviz-challenge/.venv2/lib/python3.9/site-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Clean data

## Same units

In [3]:
df["PRIX_UNITAIRE"] = df["PRIX_RELEVE"] / df["CONDITIONNEMENT"]

## Get last price

In [4]:
df[["DERNIER_PRIX", "DERNIER_PRIX_UNITAIRE", "DERNIER_DATE_RELEVE"]] = df.sort_values(by=["GENCOD", "DATE_RELEVE"], ascending=[True, True]).groupby(by=["CODE_PRODUIT", "GENCOD", "RIDET"], as_index=False)[["PRIX_RELEVE", "PRIX_UNITAIRE", "DATE_RELEVE"]].transform("last")

# Outlet selection

In [17]:
MY_GEOLOC = -22.3, 166.4
LATS = -22.35, -22.2
LNGS = 166.55, 166.36

In [23]:
df = df[(df["point_geo"].map(lambda x:float(x.split(",")[0]) if x==x else 0)<=LATS[1]) & 
   (df["point_geo"].map(lambda x:float(x.split(",")[0]) if x==x else 0)>=LATS[0]) &
   (df["point_geo"].map(lambda x:float(x.split(",")[1]) if x==x else 0)>=LNGS[1]) &
   (df["point_geo"].map(lambda x:float(x.split(",")[1]) if x==x else 0)<=LNGS[0])]

# Product selection

In [24]:
# Search in SOUS_FAMILLE
LISTE_DE_COURSES = ["pain de mie", "pâte à tartiner", 
                    "yaourt laitier", "biscuit fourré, nappé", 
                    "tablette de chocolat", 
                    "pâte longue", 'pâte courte', "pâte fraîche"]

In [37]:
df.loc[1035844]

CODE_PRODUIT                                                46241
NOM                         chocolat à tartiner Noisety pot 240 g
SECTEUR_CONSO                                     épicerie sucrée
SOUS_SECTEUR_CONSO               confiture, pâte à tartiner, miel
FAMILLE_PRODUIT                            pâte, crème à tartiner
SOUS_FAMILLE                                      pâte à tartiner
VARIETE                               pâte à tartiner au chocolat
CATEGORIE                                                   LOCAL
DEPARTEMENT                                                   Sec
MARQUE                                                  Biscochoc
CONDITIONNEMENT                                              0.24
COND_UNITE_LIBELLE_LONG                                Kilogramme
COND_UNITE_LIBELLE_COURT                                       kg
GENCOD                                3328861502400,3328860005407
PRIX_RELEVE                                                 460.0
DATE_RELEV

In [32]:
df_test = df[df["SOUS_FAMILLE"]==LISTE_DE_COURSES[1]].groupby(["CODE_PRODUIT", "NOM", "MARQUE"], as_index=False).agg(mean_price = pd.NamedAgg("DERNIER_PRIX_UNITAIRE", "mean"), min_price = pd.NamedAgg("DERNIER_PRIX_UNITAIRE", "min"), max_price = pd.NamedAgg("DERNIER_PRIX_UNITAIRE", "max")).sort_values("NOM")
df_test

,CODE_PRODUIT,NOM,MARQUE,mean_price,min_price,max_price
6,136102,beurre de cacahuète Best creamy pot 340 g,Best,1497.840655,911.764706,1823.529412
15,207557,beurre de cacahuète Best crunchy pot 340 g,Best,1288.123402,911.764706,1588.235294
4,46247,beurre de cacahuète Crown chunky pot 340 g,Crown,1286.155818,1058.823529,1676.470588
9,138680,beurre de cacahuètes Sanitarium peanut butter ...,Sanitarium,1382.115942,1288.000000,1520.000000
10,138691,beurre de cacahuètes Sanitarium smooth 375 g,Sanitarium,1385.295648,1293.333333,1520.000000
12,183059,cacao à tartiner Leader Price Pralina aux nois...,Leader Price,1933.333333,1933.333333,1933.333333
13,191699,chocolat à tartiner Banania pot 400 g,Banania,2287.500000,2287.500000,2287.500000
16,232474,chocolat à tartiner Casino Délices aux noisett...,Casino,3793.119266,3750.000000,3850.000000
17,245881,chocolat à tartiner Milka noisette/cacao pot 3...,Milka,3384.328358,1617.647059,3676.470588
3,46241,chocolat à tartiner Noisety pot 240 g,Biscochoc,2170.736755,1166.666667,3000.000000


In [41]:
import plotly.express as px

fig = px.histogram(df_test, x="NOM", y="min_price",
             barmode='group',
             height=400)
fig.show()

In [40]:
df[(df["CODE_PRODUIT"]==138635) & (df["PRIX_UNITAIRE"]<1735)][["RAISON_SOCIALE", "COMMUNE", "PRIX_UNITAIRE", "PROMOTION"]]

,RAISON_SOCIALE,COMMUNE,PRIX_UNITAIRE,PROMOTION
697417,FANAO,Nouméa,1733.333333,0
744157,CASINO PORT PLAISANCE - SUPERMARCHE,Nouméa,1726.666667,1
744175,VIVAL KOUTIO,Dumbéa,1726.666667,1
3130976,CASINO LES HALLES MAGENTA,Nouméa,1726.666667,1
3131056,CASINO PORT PLAISANCE - SUPERMARCHE,Nouméa,1726.666667,1
3131136,VIVAL KOUTIO,Dumbéa,1726.666667,1
3574160,GEANT DUMBEA - CENTRE COMMERCIAL DUMBEA MALL,Dumbéa,1726.666667,1
3574194,CASINO LES HALLES MAGENTA,Nouméa,1726.666667,1
3574236,GEANT SAINTE-MARIE,Nouméa,1726.666667,1
3574332,GEANT DUMBEA - CENTRE COMMERCIAL DUMBEA MALL,Dumbéa,1726.666667,1


In [7]:
df_test = df[df["SOUS_FAMILLE"]==LISTE_DE_COURSES[0]].groupby(["RIDET", "RAISON_SOCIALE", "CODE_PRODUIT", "NOM"], as_index=False)["DERNIER_PRIX_UNITAIRE"].agg("mean")
print(df_test["RAISON_SOCIALE"].unique())
df_test.head()

['SHELL PACIFIQUE PETROLEUM' 'SUPER U MAGECO' 'GEANT SAINTE-MARIE'
 'DISCOUNT TRIANON' 'MINI MARCHE' 'STATION SHELL VICTOIRE'
 'LIBRE SERVICE LE PACIFIQUE' 'MAGASIN G. CREUGNET' 'LA HUCHE'
 'SUPERETTE 68' 'CARREFOUR KENU IN' 'CASINO PORT PLAISANCE - SUPERMARCHE'
 'TA & TY' 'AH YEN' 'LIBRE SERVICE SAINT MICHEL' 'DAINA'
 'LIBRE SERVICE DE PLUM' 'CHEZ ALICE' 'CASINO LES HALLES MAGENTA'
 'VIVAL KOUTIO' 'STATION SHELL PAITA' 'AGRICAL' 'SUPER U KOUMAC'
 'STATION TOTAL ROBINSON' 'JOHNSTON' 'AUCHAN SUPERMARCHE TONTOUTA'
 "COTE D'ASIE" 'LA COTE' 'GOLF WABAO' 'CASINO MONT DORE'
 'ETABLISSEMENTS ROES' 'LIBRE SERVICE TOMO' 'SUPER IAORA'
 'CARREFOUR MARKET ALMA (ANCIEN CHAMPION)' 'CARREFOUR MARKET DUCOS'
 "CARREFOUR MARKET N'GEA" 'CARREFOUR MARKET ROBINSON'
 'CARREFOUR MARKET MAGENTA' 'CARREFOUR MARKET Orphelinat' 'MAXI MARKET'
 'STATION SHELL LA FOA' 'LEADER PRICE MAGENTA'
 'GEANT DUMBEA - CENTRE COMMERCIAL DUMBEA MALL'
 'LEADER PRICE PLEXUS (DUCOS)' 'FEU VERT / 218 m2' 'MAGENTA DISCOUNT'
 'LEADER

,CODE_PRODUIT,NOM,SECTEUR_CONSO,SOUS_SECTEUR_CONSO,FAMILLE_PRODUIT,SOUS_FAMILLE,VARIETE,CATEGORIE,DEPARTEMENT,MARQUE,...,NOM_VOIE,COMPLEMENT_ADRESSE,QUARTIER,CODE_POSTAL,COMMUNE,point_geo,PRIX_UNITAIRE,DERNIER_PRIX,DERNIER_PRIX_UNITAIRE,DERNIER_DATE_RELEVE
0,54548,sirop Carrefour pêche bouteille 75 cl,boisson,boisson sans alcool,sirop et concentré,sirop aromatisé,sirop de pêche,IMPORT,Sec,Carrefour,...,NaN,NaN,NaN,NaN,Païta,"-22.129734, 166.367278",846.666667,565.0,753.333333,2023-08-16
1,53350,soda aux fruits Fanta orange bouteille 1.5 l,boisson,boisson sans alcool,soda,soda aux fruits,soda aux fruits,LOCAL,Sec,Fanta,...,NaN,NaN,NaN,NaN,Poindimié,"-20.936545, 165.333038",330.000000,495.0,330.000000,2023-08-27
2,53350,soda aux fruits Fanta orange bouteille 1.5 l,boisson,boisson sans alcool,soda,soda aux fruits,soda aux fruits,LOCAL,Sec,Fanta,...,lot 3 pie section poamboa,NaN,NaN,NaN,Koné,"-21.100769, 164.896561",310.000000,475.0,316.666667,2023-08-22
3,232073,jus de fruits U orange d'Amérique brique 1 l,boisson,boisson sans alcool,jus de fruits et légumes,jus de fruits,"jus d'orange, agrume",IMPORT,Sec,U,...,Edmond Harbulot Lotissement centre Auteuil,NaN,Auteuil,NaN,Dumbéa,"-22.21118, 166.4796",295.000000,295.0,295.000000,2023-03-01
4,53350,soda aux fruits Fanta orange bouteille 1.5 l,boisson,boisson sans alcool,soda,soda aux fruits,soda aux fruits,LOCAL,Sec,Fanta,...,BONAPARTE,NaN,Rivière Salée,NaN,Nouméa,"-22.23368, 166.464522",283.333333,425.0,283.333333,2023-08-22


In [8]:
import plotly.express as px

fig = px.violin(df_test, y="DERNIER_PRIX_UNITAIRE", x="RAISON_SOCIALE", box=True, points="all",
          hover_data=df_test.columns)
fig.show()

In [40]:
df.groupby(by=["SOUS_FAMILLE"], as_index=False)["CODE_PRODUIT"].count().sort_values(by="CODE_PRODUIT", ascending=False).head(20)

,SOUS_FAMILLE,CODE_PRODUIT
147,jus de fruits,173405
13,"biscuit fourré, nappé",172079
64,conserve de légume,162496
69,conserve de poisson,137039
290,yaourt laitier,114384
231,sauce exotique,100705
12,biscuit apéritif,84847
54,condiment,76356
46,"chips, tuile et tortilla",75071
15,"biscuit sec, sablé",74448


In [42]:
df[df["NOM"].str.contains("pâte", case=False)==True]["SOUS_FAMILLE"].unique()

array(['plat cuisiné individuel', 'pâte longue', 'pâte courte',
       'pâtisserie à partager surgelée', 'pâte surgelée boulangerie',
       'pizza surgelée : fromage',
       'pâte à tarte, pizza, feuille de brick fraîche',
       'farine spécifique', 'condiment', 'aide à la pâtisserie',
       'pâte fraîche', 'olive et tartinable', 'biscuit sec, sablé',
       'traiteur : salade', 'biscuit fourré, nappé',
       'fromage : brie, camembert, coulommiers, crémeux', 'pâte sèche',
       'lasagne, cannelloni', 'pizza surgelée : végétarienne',
       'pâte fourrée', 'pâte asiatique', 'pizza surgelée : viande',
       'tarte salée surgelée : autres', 'épice culinaire',
       'chocolat à offrir', 'tablette de chocolat', 'pâte à tartiner',
       'pizza surgelée : classique', 'glace à partager : chocolat'],
      dtype=object)

In [15]:
df[df["NOM"].str.contains("coca", case=False)==True][["NOM", "CODE_PRODUIT"]].drop_duplicates().sort_values("CODE_PRODUIT")

,NOM,CODE_PRODUIT
1061,soda au cola Coca-Cola light bouteille 4 x 1.5 l,45703
979686,soda au cola Coca-Cola light boîte 33 cl,45704
1043081,soda au cola Coca-Cola light bouteille 1.5 l,45706
521890,soda au cola Coca-Cola zero bouteille 60 cl,46183
6760,soda au cola Coca-Cola bouteille 1.5 l,46185
166759,soda au cola Coca-Cola boîte 33 cl,46186
1197933,soda au cola Coca-Cola bouteille 60 cl,46187
675595,soda au cola Coca-Cola bouteille 12 x 1.5 l,53255
258648,soda au cola Coca-Cola boîte 24 x 33 cl,53258
552615,soda au cola Coca-Cola zero boîte 6 x 33 cl,53325
